In [9]:
import numpy as np
import numpy
from pyscf import gto, scf, cc, ao2mo,lib

mol = gto.M(    atom = 'He 0 0 0',basis = 'def2-QZVPP')
mf = scf.RHF(mol).run()
print (mf.make_rdm1())
mycc = cc.CCSD(mf).run()

# CCSD density matrix in MO basis
dm1 = mycc.make_rdm1()
dm2 = mycc.make_rdm2()

# CCSD energy based on density matrices
h1 = np.einsum('pi,pq,qj->ij', mf.mo_coeff.conj(), mf.get_hcore(), mf.mo_coeff)
nmo = mf.mo_coeff.shape[1]
#Electron Repulsion Integrals
eri = ao2mo.kernel(mol,mf.mo_coeff,compact=False).reshape([nmo]*4)
E = np.einsum('pq,qp', h1, dm1)

# Note dm2 is transposed to simplify its contraction to integrals
E+= np.einsum('pqrs,pqrs', eri, dm2)*.5
E+= mol.energy_nuc()
print('E(CCSD) = %s, reference %s' % (E, mycc.e_tot))

# When plottingdm2 CCSD density on the grids, CCSD density matrices need to be
# first transformed to AO basis.
dm1_ao=np.einsum('pi,ij,qj->pq', mf.mo_coeff, dm1, mf.mo_coeff.conj())

converged SCF energy = -2.86162483917628
[[ 9.42320761e-002  1.44033347e-001  1.82393252e-001  8.09740958e-002
  -1.94540609e-031  7.10199597e-033 -1.33282363e-047 -3.85448681e-031
   5.50309535e-033  5.53309468e-047 -2.74582850e-031  1.32163700e-033
   1.46263777e-046  1.00894568e-064 -4.04358057e-079 -1.55147988e-017
   7.78440421e-078 -1.71227547e-062 -5.10131308e-065 -4.89507941e-079
   4.03707374e-017  8.59760393e-077 -7.56233499e-062 -2.14051823e-049
   1.03141964e-109 -1.36693200e-049  1.09608024e-062 -3.55048784e-048
  -8.78064216e-108  3.20890911e-047]
 [ 1.44033347e-001  2.20154388e-001  2.78787348e-001  1.23768578e-001
  -2.97354533e-031  1.08553721e-032 -2.03721553e-047 -5.89156749e-031
   8.41145898e-033  8.45731284e-047 -4.19698775e-031  2.02011680e-033
   2.23563591e-046  1.54216938e-064 -6.18059654e-079 -2.37143071e-017
   1.18984303e-077 -2.61720613e-062 -7.79733640e-065 -7.48210908e-079
   6.17065088e-017  1.31414028e-076 -1.15589984e-061 -3.27177346e-049
   1.5765207

E(CCSD) = -2.902451379322164  E_corr = -0.0408265401458826
E(CCSD) = -2.902451415735089, reference -2.902451379322164


In [5]:
dm1=mycc.make_rdm1(ao_repr=False)

In [11]:
mf.mo_coeff.shape

(30, 30)

In [37]:
# t1,t2 are the excitator operators wether l1,l2 are the deexcitator operators
def _gamma1_intermediates(mycc, t1, t2, l1, l2):
    nocc, nvir = t1.shape
    doo =-numpy.einsum('ja,ia->ij', t1, l1)
    dvv = numpy.einsum('ia,ib->ab', t1, l1)
    xtv = numpy.einsum('ie,me->im', t1, l1)
    dvo = t1.T - numpy.einsum('im,ma->ai', xtv, t1)
    theta = t2 * 2 - t2.transpose(0,1,3,2)
    doo -= lib.einsum('jkab,ikab->ij', theta, l2)
    dvv += lib.einsum('jica,jicb->ab', theta, l2)
    xt1  = lib.einsum('mnef,inef->mi', l2, theta)
    xt2  = lib.einsum('mnaf,mnef->ea', l2, theta)
    dvo += numpy.einsum('imae,me->ai', theta, l1)
    dvo -= numpy.einsum('mi,ma->ai', xt1, t1)
    dvo -= numpy.einsum('ie,ae->ai', t1, xt2)
    dov = l1
    return doo, dov, dvo, dvv

In [6]:
mf.mo_coeff.dot(np.array([[2,0],[0,0]]).dot(mf.mo_coeff.T))

array([[0.70112044, 0.60816931],
       [0.60816931, 0.52754118]])

In [44]:
# copy paste of source code
def _make_rdm1(mycc, d1, with_frozen=True, ao_repr=False):
    '''dm1[p,q] = <q_alpha^\dagger p_alpha> + <q_beta^\dagger p_beta>
    The contraction between 1-particle Hamiltonian and rdm1 is
    E = einsum('pq,qp', h1, rdm1)
    doo=t1  dov=t2   dvo=l1   dvv=l2
    '''
    doo, dov, dvo, dvv = d1
    nocc, nvir = dov.shape
    nmo = nocc + nvir
    dm1 = numpy.empty((nmo,nmo), dtype=doo.dtype)
    dm1[:nocc,:nocc] = doo + doo.conj().T
    dm1[:nocc,nocc:] = dov + dvo.conj().T
    dm1[nocc:,:nocc] = dm1[:nocc,nocc:].conj().T
    dm1[nocc:,nocc:] = dvv + dvv.conj().T
    dm1[numpy.diag_indices(nocc)] += 2
    return dm1

In [47]:
mycc.t1, mycc.t2,mycc.l1, mycc.l2

(array([[0.00208455]]),
 array([[[[-0.06589654]]]]),
 array([[0.00193877]]),
 array([[[[-0.06560679]]]]))

In [9]:
mycc.t2*2, mycc.t1*2

(array([[[[-0.13179307]]]]), array([[0.0041691]]))

In [20]:
dm1[np.diag_indices(1)] -= 2

In [46]:
dm1

array([[1.9913454 , 0.00387753],
       [0.00387753, 0.0086546 ]])

In [45]:
_make_rdm1(mycc,_gamma1_intermediates(mycc, t1, t2, l1, l2))

array([[1.9913454 , 0.00387753],
       [0.00387753, 0.0086546 ]])